In [431]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz
import string
from googletrans import Translator # version: pip install googletrans==3.1.0a0

In [432]:
data =pd.read_excel('Ozurgeti.xlsx')                   

In [433]:
data.head(4)
data.columns

Index(['საიდენტიფიკაციო ნომერი', 'პირადი ნომერი',
       'ორგანიზაციულ-სამართლებრივი ფორმა', 'სუბიექტის დასახელება',
       'რეგიონი (იურიდიული)', 'მუნიციპალიტეტი/ქალაქი (იურიდიული)',
       'დაბის/თემის/სოფლის საკრებულო (იურიდიული)', 'დაბა/სოფელი (იურიდიული)',
       'იურიდიული მისამართი', 'რეგიონი (ფაქტობრივი)',
       'მუნიციპალიტეტი/ქალაქი (ფაქტობრივი)',
       'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)', 'დაბა/სოფელი (ფაქტობრივი)',
       'ფაქტობრივი მისამართი', 'საქმიანობის კოდი NACE Rev.2',
       'საქმიანობის დასახელება NACE Rev.2', 'პარტნიორი', 'ხელმძღვანელი',
       'ტელეფონი', 'ელფოსტა', 'საკუთრების ფორმა',
       'აქტიური ეკონომიკური სუბიექტები', 'ბიზნესის ზომა'],
      dtype='object')

In [434]:
data.head(3)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,რეგიონი (იურიდიული),მუნიციპალიტეტი/ქალაქი (იურიდიული),დაბის/თემის/სოფლის საკრებულო (იურიდიული),დაბა/სოფელი (იურიდიული),იურიდიული მისამართი,რეგიონი (ფაქტობრივი),...,ფაქტობრივი მისამართი,საქმიანობის კოდი NACE Rev.2,საქმიანობის დასახელება NACE Rev.2,პარტნიორი,ხელმძღვანელი,ტელეფონი,ელფოსტა,საკუთრების ფორმა,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,იმ,ზურაბ შელეგია,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,ქ. თბილისის ნაძალადევის რაიონში / მანჯგალაძის 29,გურია,...,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",ზურაბ შელეგია - 100,ზურაბ შელეგია,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
1,100019383,6.200100e+10,იმ,მარინე ჭელიძე,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,"ქ. თბილისის ნაძალადევის რაიონში / სანზონა კ.6,ბ.1",გურია,...,NaN,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",მარინე ჭელიძე - 100,მარინე ჭელიძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
2,100085300,3.300100e+10,იმ,დავით ანთაძე,გურია,ოზურგეთის მუნიციპალიტეტი,ბახვის თემის საკრებულო,NaN,ბახვის თემის საკრებულო,გურია,...,ბახვის თემის საკრებულო,NaN,NaN,დავით ანთაძე - 100,დავით ანთაძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN


In [435]:
data = data[['საიდენტიფიკაციო ნომერი',
             'პირადი ნომერი',
             'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)',
             'ორგანიზაციულ-სამართლებრივი ფორმა',
             'სუბიექტის დასახელება','ფაქტობრივი მისამართი',
             'საქმიანობის დასახელება NACE Rev.2',
             'აქტიური ეკონომიკური სუბიექტები',
             'ბიზნესის ზომა']]

In [436]:
data.head(4)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,NaN,იმ,ზურაბ შელეგია,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
1,100019383,6.200100e+10,ურეკის სადაბო საკრებულო,იმ,მარინე ჭელიძე,NaN,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
2,100085300,3.300100e+10,ბახვის თემის საკრებულო,იმ,დავით ანთაძე,ბახვის თემის საკრებულო,NaN,NaN,NaN
3,100096389,3.300101e+10,NaN,იმ,სოსო სადრაძე,შემოქმედის,"სხვა ხილის, კენკროვნების და კაკლისებრების მოყვანა",NaN,NaN


In [437]:
data = data[(data['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)'].str.contains('ქ. ოზურგეთი', na=False)) ]


In [438]:
data = data[~(data['ფაქტობრივი მისამართი'].isnull())]

In [439]:
def clean_text(arg):
    cleaned = re.sub(r'[^\w\s]',' ',arg)
    return cleaned


def remove_punctuation(input_string):
    result = input_string
    for char in string.punctuation:
        result = result.replace(char, ' ')
    return result


In [440]:
data['ნომერი'] = data['ფაქტობრივი მისამართი'].apply(lambda x: "".join([i for i in x if i.isnumeric()]))

In [441]:
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(clean_text)
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(remove_punctuation)

In [442]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: "".join([re.sub('\d+', '',i)  for i in x ]))

In [443]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: "".join([re.sub(r'\\N', '',i)  for i in x ]))

In [444]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ''.join([i for i in x if i != 'N']))

In [445]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i not in [ 'ქუჩა',
                                                                                        'საქართველო',
                                                                                        'ოზურგეთი',
                                                                                        'ოზურგეთის',
                                                                                        'რაიონში',
                                                                                        'სახელობის',
                                                                                        'შესახვევი',
                                                                                        'შესახ',
                                                                                        'ჩიხი'
                                                                                        ]
                                                                                    ]
                                                                                )
                                                                            )

In [446]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i.isnumeric() or len(i) >3]))

In [447]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: x.replace('თაყაიშვილის', 'საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი') if 'თაყაიშვილის' in x else x)

In [448]:
data = data[ (data['ქუჩა']!='')]
print(data.shape)

(788, 11)


In [449]:
data = data[(data['ნომერი']!='')]
print(data.shape)

(713, 11)


In [450]:
data.sort_values(by='ქუჩა', ascending=True)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა
6470,33001079313,3.300108e+10,ქ. ოზურგეთი,იმ,ლევანი ბერიძე,აბაშიძის ქუჩა #9,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,9,აბაშიძის
4724,33001025731,3.300103e+10,ქ. ოზურგეთი,იმ,ცისანა ფრანგულაშვილი,აბაშიძის ქუჩა #6,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,6,აბაშიძის
5975,33001066269,3.300107e+10,ქ. ოზურგეთი,იმ,დავით ბაბილოძე,აბაშიძის 5,"საცალო ვაჭრობა რკინა-კავეულით, საღებავებით და ...",NaN,NaN,5,აბაშიძის
1978,237108786,NaN,ქ. ოზურგეთი,შპს,გეოაგროინვესტი,აკაკი წერეთლის 29,NaN,NaN,NaN,29,აკაკი წერეთლის
1795,237058447,NaN,ქ. ოზურგეთი,სს,სს ოზურგეთლუდი,აკაკის ქ. №29,საკუთარი ან იჯარით აღებული არასაცხოვრებელი შენ...,აქტიური,მცირე,29,აკაკის
...,...,...,...,...,...,...,...,...,...,...,...
4188,33001016395,3.300102e+10,ქ. ოზურგეთი,იმ,ზაზა გოგუაძე,ჯორჯიაშვილის ქუჩა #44,იატაკის დაგება და კედლის მოსაპირკეთებელი სამუშ...,აქტიური,მცირე,44,ჯორჯიაშვილის
4704,33001025448,3.300103e+10,ქ. ოზურგეთი,იმ,ირინა კაპანაძე,ჯორჯიაშვილის ქ. 42,სხვა სახის მეფრინველეობა,აქტიური,მცირე,42,ჯორჯიაშვილის
344,137066029,3.300102e+10,ქ. ოზურგეთი,იმ,დავით გაბედავა,ჯორჯიაშვილის 40,მიწის ნაკვეთების იჯარით გაცემა,NaN,NaN,40,ჯორჯიაშვილის
2045,237113333,NaN,ქ. ოზურგეთი,შპს,შპს სალი 2009,"ჯორჯიაშვილის ქუჩა, №12, ბ 16",საბითუმო ვაჭრობა თამბაქოს პროდუქტებით,აქტიური,მცირე,1216,ჯორჯიაშვილის


In [451]:
def translate_text(text, dest_language='en'): 
    translator = Translator()
    result = translator.translate(text, dest=dest_language)
    return result.text

In [452]:
# data['translated_street']=data['ქუჩა'].apply(translate_text)

In [453]:
data.head(3)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა
53,102398219,3.300103e+10,ქ. ოზურგეთი,იმ,ელისო ქადარია,ბათუმის 3,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,3,ბათუმის
83,105211994,1.007003e+09,ქ. ოზურგეთი,იმ,მადალენა თოიძე,გურამიშვილის ქ.12,რესტორნები და საკვებით მობილური მომსახურების ს...,NaN,NaN,12,გურამიშვილის
141,131951583,2.100100e+10,ქ. ოზურგეთი,იმ,ზაალი ფანჩულიძე,ე. თაყაიშვილის I შეს. N1,უალკოჰოლო არომატიზებული და/ან დამტკბარი სასმელ...,NaN,NaN,1,საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი


In [454]:
def fuzzy_match(ops_name, customer_input):
    similarity_score = fuzz.ratio(ops_name, customer_input)
    return similarity_score, customer_input

In [455]:
OPS_Streets = pd.read_csv('Ozurgeti_streets.txt',header =None, names=['street_name'])
OPS_Streets['street_name'] = OPS_Streets['street_name'].apply(lambda x: x.replace('ქუჩა','') if 'ქუჩა' in x.split() else x)
OPS_Streets = OPS_Streets[~OPS_Streets['street_name'].str.contains(r'\b(?:ჩიხი|შესახვევი)\b', regex=True)]

In [456]:
def match_ops_street(x):
    for i in OPS_Streets['street_name']:
        if x.strip() in i.split() and len(x.strip()) < len(i.strip()):
            return i
    return None 

data['OPS'] = data['ქუჩა'].apply(match_ops_street)


In [457]:
data.head(2)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა,OPS
53,102398219,3.300103e+10,ქ. ოზურგეთი,იმ,ელისო ქადარია,ბათუმის 3,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,3,ბათუმის,None
83,105211994,1.007003e+09,ქ. ოზურგეთი,იმ,მადალენა თოიძე,გურამიშვილის ქ.12,რესტორნები და საკვებით მობილური მომსახურების ს...,NaN,NaN,12,გურამიშვილის,დავით გურამიშვილის


In [458]:
data['Similarity'] = data['ქუჩა'].apply(
    lambda x: max([fuzzy_match(x, street) for street in OPS_Streets['street_name']])
)

In [459]:
data[['similarity_score', 'matched_street']] = pd.DataFrame(data['Similarity'].tolist(), index=data.index)


In [460]:
data.drop(columns='Similarity', inplace=True)

In [461]:
data.head(4)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა,OPS,similarity_score,matched_street
53,102398219,3.300103e+10,ქ. ოზურგეთი,იმ,ელისო ქადარია,ბათუმის 3,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,3,ბათუმის,None,93.333333,ბათუმის
83,105211994,1.007003e+09,ქ. ოზურგეთი,იმ,მადალენა თოიძე,გურამიშვილის ქ.12,რესტორნები და საკვებით მობილური მომსახურების ს...,NaN,NaN,12,გურამიშვილის,დავით გურამიშვილის,77.419355,დავით გურამიშვილის
141,131951583,2.100100e+10,ქ. ოზურგეთი,იმ,ზაალი ფანჩულიძე,ე. თაყაიშვილის I შეს. N1,უალკოჰოლო არომატიზებული და/ან დამტკბარი სასმელ...,NaN,NaN,1,საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი,None,100.000000,საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი
165,137059778,3.300101e+10,ქ. ოზურგეთი,იმ,ნაირა მეგრელიშვილი,გურიის ქ.N33,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",აქტიური,მცირე,33,გურიის,None,92.307692,გურიის


In [462]:
def fill_ops(row):
    if pd.isna(row['OPS']):
        return row['matched_street']
    else:
        return row['OPS']

data['OPS'] = data.apply(fill_ops, axis=1)


In [463]:
data['OPS'] = data['OPS'].apply(str.strip)

In [464]:
data['St_Full_Name']=data['ნომერი'] + ' ' + data['matched_street'] + ' ქუჩა, Ozurgeti, Georgia'


In [465]:
data.head(3)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა,OPS,similarity_score,matched_street,St_Full_Name
53,102398219,3.300103e+10,ქ. ოზურგეთი,იმ,ელისო ქადარია,ბათუმის 3,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,აქტიური,მცირე,3,ბათუმის,ბათუმის,93.333333,ბათუმის,"3 ბათუმის ქუჩა, Ozurgeti, Georgia"
83,105211994,1.007003e+09,ქ. ოზურგეთი,იმ,მადალენა თოიძე,გურამიშვილის ქ.12,რესტორნები და საკვებით მობილური მომსახურების ს...,NaN,NaN,12,გურამიშვილის,დავით გურამიშვილის,77.419355,დავით გურამიშვილის,"12 დავით გურამიშვილის ქუჩა, Ozurgeti, Georgia"
141,131951583,2.100100e+10,ქ. ოზურგეთი,იმ,ზაალი ფანჩულიძე,ე. თაყაიშვილის I შეს. N1,უალკოჰოლო არომატიზებული და/ან დამტკბარი სასმელ...,NaN,NaN,1,საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი,საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი,100.000000,საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი,1 საჯავახო — ჩოხატაური — ოზურგეთი — ქობულეთი ქ...


In [ ]:
# data.to_excel('Ozurgeti_Steet_WithFuzzy.xlsx', index=False)

________________

** რაც საზღვრის მიღმაა დავიკიდოთ

** ქუჩის პარამეტრები
